In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
# model = AutoModelForCausalLM.from_pretrained(
#     "openai/gpt-oss-20b",
#     torch_dtype=torch.bfloat16,
# ).to('cuda:1')



/home/tikhonov/.conda/envs/condaenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/tikhonov/.conda/envs/condaenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [6]:
messages = [
    {"role": "system", "content": "YOYOYOYOYUOY"},
    {"role": "user", "content": "341*6432"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
)

print(repr(tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=False)))

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-09-17\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions\n\nYOYOYOYOYUOY\n\n<|end|><|start|>user<|message|>341*6432<|end|><|start|>assistant'


In [ ]:
# 1. Формируем историю с помощью apply_chat_template
messages = [

    {"role": "user", "content": "341*6432"}
]

# Получаем строку со всеми системными промптами и историей, но без промпта для генерации
prompt_history = tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)

# 2. Вручную добавляем "подставные" мысли, НЕ закрывая теги
# Возьмем часть рассуждений из первого рабочего примера
qwen_reasoning_part = "We are a language model. The user gives a multiplication problem \"341*6432\". They likely want the product. So compute 341 *"
manual_part = f"<|start|>assistant<|channel|>analysis<|message|>{qwen_reasoning_part}"

full_prompt_string = prompt_history + manual_part

print("--- Manually constructed prompt string ---")
print(repr(full_prompt_string))
print("\n--- Tokenizing and generating ---")

# 3. Токенизируем и генерируем
inputs = tokenizer(full_prompt_string, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=2048)

# 4. Декодируем только новую, сгенерированную часть
generated_part = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=False)

print("\n--- Model Generation (what it added) ---")
print(generated_part)



--- Manually constructed prompt string ---
'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-09-17\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>341*6432<|end|><|start|>assistant<|channel|>analysis<|message|>We are a language model. The user gives a multiplication problem "341*6432". They likely want the product. So compute 341 *'

--- Tokenizing and generating ---

--- Model Generation (what it added) ---
 6432. Let's do it manually: 6432 * 341 = 6432 * 300 + 6432 * 40 + 6432 * 1. 6432*300 = 1,929,600 (since 6432*100=643200, times 3). 6432*40 = 257,280. 6432 = 6,432. Sum: 1,929,600 + 257,280 = 2,186,880. Then add 6,432: Total 2,193,312. Alternatively more precisely: Let's multiply: 6432 * 341. Check: 6432 * 341: 6432 * (300 + 40 + 1) as above. 6432*300=1,929,600. 6432*40=257,280. 6432*1=6,432. Add: 1,929,6

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm.auto import tqdm



tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-70B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-70B", dtype=torch.bfloat16, device_map='auto')


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

['140', '1']

In [3]:

tokenizer.tokenize('14010')

['140', '10']

In [6]:

messages = [
    {"role": "system", "content": "Answer only with the number. Don't write any explanations."},
    {"role": "user", "content": "23+17"}
]


prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
print(str(prompt))


output = model.generate(tokenizer(prompt, return_tensors='pt').input_ids.to(model.device), max_new_tokens=2048)

print(tokenizer.decode(output[0], skip_special_tokens=False))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<｜begin▁of▁sentence｜>Answer only with the number. Don't write any explanations.<｜User｜>23+17<｜Assistant｜><think>

<｜begin▁of▁sentence｜><｜begin▁of▁sentence｜>Answer only with the number. Don't write any explanations.<｜User｜>23+17<｜Assistant｜><think>
I need to add the numbers 23 and 17 together.

First, I'll add the units digits: 3 plus 7 equals 10.

Next, I'll add the tens digits: 2 plus 1 equals 3.

Finally, I'll combine these results to get the total sum.
</think>

23 + 17 equals 40.

**Step-by-Step Explanation:**

1. **Add the units place:**
   
   \[
   3 \ (\text{from } 23) + 7 \ (\text{from } 17) = 10
   \]

2. **Add the tens place:**
   
   \[
   2 \ (\text{from } 23) + 1 \ (\text{from } 17) = 3
   \]

3. **Combine the results:**
   
   \[
   3 \ (\text{tens}) \times 10 + 0 \ (\text{units}) = 40
   \]

**Final Answer:** \boxed{40}<｜end▁of▁sentence｜>


In [10]:
tokenizer.apply_chat_template(
    [{"role": "user", "content": "23+17"}, {"role": "assistant", "content": "<think>test thinking</THINK>40"}],
    tokenize=False,
    add_generation_prompt=False
).replace(
 
    '</THINK>', '<think>'
)

'<｜begin▁of▁sentence｜><｜User｜>23+17<｜Assistant｜><think>test thinking<think>40<｜end▁of▁sentence｜>'